<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/helgeingvart/Transformer-timeseries/blob/master/chad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [ ]:
import torch
import torch.nn as nn

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, n_classes):
        super(TimeSeriesTransformer, self).__init__()

        # Input projection from num_features to d_model
        self.input_projection = nn.Linear(num_features, d_model)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, d_model))  # Assuming max seq_len of 1000

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)

        # Output MLP for classification
        # self.fc = nn.Sequential(  
        #               nn.Linear(d_model, 512), 
        #               nn.ReLU(), 
        #               nn.Linear(512, 256),
        #               nn.ReLU(), 
        #               nn.Linear(256, 128), 
        #               nn.ReLU(), 
        #               nn.Linear(128, n_classes))
        # 
        self.fc = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.ReLU(),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):
        # Input: (batch_size, seq_len, num_features)

        batch_size, seq_len, _ = x.size()

        # Input projection (embedding)
        x = self.input_projection(x)  # (batch_size, seq_len, d_model)

        # Add positional encoding. This code supports various lengths of the time-series (seq_len might vary)
        x = x + self.positional_encoding[:, :seq_len, :]

        # Transpose for Transformer (needed by PyTorch's nn.Transformer)
        x = x.transpose(0, 1)  # (seq_len, batch_size, d_model)

        # Transformer Encoder
        x = self.transformer_encoder(x)  # (seq_len, batch_size, d_model)

        # Pooling: Take the mean over the sequence dimension
        x = x.mean(dim=0)  # (batch_size, d_model)

        # Classification MLP
        out = self.fc(x)  # (batch_size, n_classes)

        return out



## Datasets and dataloader classes for training, validation and testing


In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import pandas as pd
import zipfile

zip_file = 'data/mitbih.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extract('mitbih_test.csv', 'data')
    zip_ref.extract('mitbih_train.csv', 'data')

# train_data = 'data/mitbih_train.csv'
df_train = pd.read_csv('data/mitbih_train.csv')
df_test = pd.read_csv('data/mitbih_test.csv')

# Custom Dataset class for time series data
class TimeSeriesDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        # Add another dimension at the end if we later on wants to work on multivariate time-series
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.features = self.features.unsqueeze(-1)  

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

batch_size = 100

#### TEST data #####
# Separate features and labels
labels = df_test.iloc[:, -1].values     # Last column as the label
features = df_test.iloc[:, :-1].values  # All columns except the last one

# Create Dataset and DataLoader for testing
dataset = TimeSeriesDataset(features, labels)
test_loader = DataLoader(dataset, batch_size, shuffle=True)

#### TRAIN/VALIDATION data ####
# Separate features and labels
labels = df_train.iloc[:, -1].values     # Last column as the label
features = df_train.iloc[:, :-1].values  # All columns except the last one

# Create Dataset and DataLoader
dataset = TimeSeriesDataset(features, labels)

# Dataset splitting for training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx+1}:")
    print(f"Inputs: {inputs.shape}")
    print(f"Targets: {targets.shape}")
    break  # Just to show the first batch


## Training
Now, running through all epochs and batches

In [ ]:
import torch.optim as optim

# Parameters
num_features = 1     # Number of features in the time series
seq_len = 187        # Length of each time series sequence
d_model = 200        # Transformer model dimension
nhead = 2            # Number of attention heads
num_encoder_layers = 1 # Number of Transformer encoder layers
dim_feedforward = 128 # Feedforward dimension
dropout = 0.1        # Dropout rate
n_classes = 5        # Number of output classes
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize model, loss function, and optimizer
model = TimeSeriesTransformer(num_features, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model = model.to(device)

# Training and Validation Loop
n_epochs = 50
best_loss = 1e10 # Used for saving model upon optimal validation loss

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}')

    # Training phase
    model.train()  # Set model to training mode
    running_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # Move data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_loader)
    print(f'Training Loss: {avg_train_loss:.4f}')

    # Validation phase
    model.eval()  # Set model to evaluation mode
    running_val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            # Accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = running_val_loss / len(val_loader)
        
    val_accuracy = correct / total
    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        print("saving best model")
        torch.save(model.state_dict(), 'myModel.pth')


## Testing
The testing should be done on the model with the least validation loss, which was saved during the training.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = TimeSeriesTransformer(num_features, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, n_classes)
# Load the saved model parameters
model.load_state_dict(torch.load('myModel.pth', weights_only=False))

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval() 

# Run inference
all_targets = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_targets.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Evaluate performance
test_accuracy = accuracy_score(all_targets, all_predictions)
print(f'Test Accuracy: {test_accuracy:.4f}')

report = classification_report(all_targets, all_predictions, digits=4)
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(all_targets, all_predictions)
print("Confusion Matrix:")
print(conf_matrix)